In [38]:
import geopandas as gpd
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier

In [5]:
change_type_map = {'Demolition': 0, 'Road': 1, 'Residential': 2, 'Commercial': 3, 'Industrial': 4,
       'Mega Projects': 5} 

In [21]:
train_df = gpd.read_file('train.geojson')
test_df = gpd.read_file('test.geojson')

# <u> Data Preprocessing and Cleaning <u/>

In [22]:
y_train=train_df['change_type'].map(change_type_map) #we convert the target variable to numeric values using the dictionary we created earlier
x_train=train_df.drop(columns=['change_type','index'])#we drop index and change_type columns from the training data
x_test=test_df.drop(columns=['index'])#we drop index column from the test data

In [23]:
numerical_cols = x_train.select_dtypes(include=["number", "bool"]).columns.to_list()
categorical_cols = x_train.select_dtypes(include=["object", "category"]).columns.to_list()

### Geometry features engineering:

In [ ]:
def add_geometry_features(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    gdf = gdf.copy()


    if gdf.crs is None:
        gdf = gdf.set_crs("EPSG:4326", allow_override=True)


    gdf_m = gdf.to_crs(epsg=3857)

    area = gdf_m.geometry.area
    perim = gdf_m.geometry.length


    gdf_ll = gdf.to_crs(epsg=4326)
    cent = gdf_ll.geometry.centroid
    bounds = gdf_ll.geometry.bounds  

    gdf["geom_area_m2"] = area
    gdf["geom_perim_m"] = perim
    gdf["geom_compactness"] = (4 * np.pi * area) / (perim**2 + 1e-9)

    gdf["centroid_lon"] = cent.x
    gdf["centroid_lat"] = cent.y

    gdf["bbox_width_deg"]  = bounds["maxx"] - bounds["minx"]
    gdf["bbox_height_deg"] = bounds["maxy"] - bounds["miny"]
    gdf["bbox_area_deg2"]  = gdf["bbox_width_deg"] * gdf["bbox_height_deg"]

    return gdf


In [ ]:
#for x_train
x_train = add_geometry_features(x_train)
x_train = x_train.drop(columns=["geometry"])

# for x_test
x_test = add_geometry_features(x_test)
x_test = x_test.drop(columns=["geometry"])

### Date features engineering:

In [ ]:
import re

def _detect_date_indices(df):
    date_cols = [c for c in df.columns if re.fullmatch(r"date\d+", c)]
    if not date_cols:
        raise ValueError("Aucune colonne dateX trouvée (date0.. ou date1..)")
    idxs = sorted([int(c.replace("date","")) for c in date_cols])
    return idxs, [f"date{i}" for i in idxs]

def add_temporal_status_image_features(df: pd.DataFrame, drop_raw: bool = True) -> pd.DataFrame:
    """
    - Trie chaque ligne par les vraies dates (NaT à la fin)
    - Construit des features de statuts (ordre + transitions)
    - Construit des features RGB (first/last/deltas) après tri
    - Ajoute aussi date_span_days + deltas entre dates
    """
    df = df.copy()

    idxs, date_cols = _detect_date_indices(df)
    k = len(date_cols)

    status_cols = [f"change_status_date{i}" for i in idxs if f"change_status_date{i}" in df.columns]
    if len(status_cols) != k:

        status_cols = [c for c in df.columns if re.fullmatch(r"change_status_date\d+", c)]
        status_idxs = sorted([int(c.split("date")[-1]) for c in status_cols])
        status_cols = [f"change_status_date{i}" for i in status_idxs]
        k = min(len(date_cols), len(status_cols))
        date_cols, status_cols, idxs = date_cols[:k], status_cols[:k], idxs[:k]


    img_specs = []
    for color in ["red", "green", "blue"]:
        for stat in ["mean", "std"]:
            cols = []
            ok = True
            for i in idxs[:k]:
                c = f"img_{color}_{stat}_date{i}"
                if c in df.columns:
                    cols.append(c)
                else:
                    ok = False
                    break
            if ok:
                img_specs.append((color, stat, cols))


    D = df[date_cols].apply(pd.to_datetime, dayfirst=True, errors="coerce").to_numpy(dtype="datetime64[ns]")
    Di = D.astype("int64")
    nat_mask = (Di == np.iinfo(np.int64).min)
    Di[nat_mask] = np.iinfo(np.int64).max

    order = np.argsort(Di, axis=1)
    Di_sorted = np.take_along_axis(Di, order, axis=1)

    maxv = np.iinfo(np.int64).max
    valid_mask = (Di_sorted != maxv)
    last_idx = valid_mask.sum(axis=1) - 1


    span_days = np.full(len(df), np.nan)
    ok = last_idx >= 0
    span_days[ok] = (Di_sorted[ok, last_idx[ok]] - Di_sorted[ok, 0]) / (24 * 3600 * 1e9)
    df["date_span_days"] = span_days

    for j in range(k - 1):
        a, b = Di_sorted[:, j], Di_sorted[:, j + 1]
        delta = (b - a) / (24 * 3600 * 1e9)
        bad = (a == maxv) | (b == maxv)
        delta[bad] = np.nan
        df[f"delta_sorted_{j}_{j+1}_days"] = delta


    S = df[status_cols].astype(object).to_numpy()
    S_sorted = np.take_along_axis(S, order[:, :k], axis=1)


    Sdf = pd.DataFrame(S_sorted)
    df["status_first"] = Sdf.bfill(axis=1).iloc[:, 0]
    df["status_last"]  = Sdf.ffill(axis=1).iloc[:, -1]

    df = pd.concat([df, pd.get_dummies(df["status_first"], prefix="status_first", dummy_na=True)], axis=1)
    df = pd.concat([df, pd.get_dummies(df["status_last"],  prefix="status_last",  dummy_na=True)], axis=1)


    level_map = {
        "Land Cleared": 0,
        "Construction Started": 1,
        "Materials Dumped": 1,
        "Construction Midway": 2,
        "Construction Done": 3,
        "Prior Construction": 3,
        "Operational": 4,
    }

    L = np.full((len(df), k), np.nan)
    for j in range(k):
        L[:, j] = pd.Series(S_sorted[:, j]).map(level_map).to_numpy(dtype=float)
        df[f"status_level_sorted_{j}"] = L[:, j]

    Ldf = pd.DataFrame(L)
    first_level = Ldf.bfill(axis=1).iloc[:, 0].to_numpy()
    last_level  = Ldf.ffill(axis=1).iloc[:, -1].to_numpy()

    df["status_level_first"] = first_level
    df["status_level_last"]  = last_level
    df["status_progress"]    = last_level - first_level
    df["status_level_max"]   = np.nanmax(L, axis=1)
    df["status_level_mean"]  = np.nanmean(L, axis=1)


    mask = ~pd.DataFrame(S_sorted).isna().to_numpy()
    n_changes = np.zeros(len(df), dtype=float)
    for j in range(k - 1):
        m = mask[:, j] & mask[:, j + 1]
        n_changes += (m & (S_sorted[:, j] != S_sorted[:, j + 1]))
    df["status_n_changes"] = n_changes

    n_reg = np.zeros(len(df), dtype=float)
    for j in range(k - 1):
        a, b = L[:, j], L[:, j + 1]
        m = np.isfinite(a) & np.isfinite(b)
        n_reg += (m & (b < a))
    df["status_n_regressions"] = n_reg
    df["status_has_regression"] = (n_reg > 0).astype(int)


    first_date = Di_sorted[:, 0]
    time_to_done = np.full(len(df), np.nan)
    for i in range(len(df)):
        if first_date[i] == maxv:
            continue
        idx_done = None
        for j in range(k):
            if valid_mask[i, j] and np.isfinite(L[i, j]) and L[i, j] >= 3:
                idx_done = j
                break
        if idx_done is not None:
            time_to_done[i] = (Di_sorted[i, idx_done] - first_date[i]) / (24 * 3600 * 1e9)
    df["time_to_done_days"] = time_to_done


    for color, stat, cols in img_specs:
        X = df[cols].to_numpy(dtype=float)
        Xs = np.take_along_axis(X, order[:, :k], axis=1)
        Xs = Xs.copy()
        Xs[~valid_mask] = np.nan

        first = Xs[:, 0]
        last = np.full(len(df), np.nan)
        ok = last_idx >= 0
        last[ok] = Xs[ok, last_idx[ok]]

        df[f"img_{color}_{stat}_first"] = first
        df[f"img_{color}_{stat}_last"]  = last
        df[f"img_{color}_{stat}_delta"] = last - first

        for j in range(k - 1):
            df[f"img_{color}_{stat}_delta_{j}_{j+1}"] = Xs[:, j + 1] - Xs[:, j]


    if drop_raw:
        drop_cols = []
        drop_cols += date_cols + status_cols
        for _, _, cols in img_specs:
            drop_cols += cols
        drop_cols += ["status_first", "status_last"]
        df = df.drop(columns=[c for c in drop_cols if c in df.columns])

    return df


In [ ]:
#create temporal features for both train and test
x_train = add_temporal_status_image_features(x_train)
x_test = add_temporal_status_image_features(x_test) 

### Multi-hot encoding: urban_type & geography_type

In [ ]:
def multi_hot_encode(df: pd.DataFrame, column: str, all_values: set = None) -> tuple:
    """
    Crée un encodage multi-hot pour une colonne avec des valeurs séparées par des virgules.
    
    Args:
        df: DataFrame avec la colonne à encoder
        column: Nom de la colonne à encoder
        all_values: Ensemble des valeurs possibles (optionnel). Si None, extrait du df.
    
    Returns:
        tuple: (DataFrame encodé, set des valeurs uniques)
    """
    df = df.copy()
    

    if all_values is None:
        all_values = set()
        for val in df[column].dropna():
            if pd.notna(val) and val != 'N,A': 

                values = [v.strip() for v in str(val).split(',')]
                all_values.update(values)
    

    for value in sorted(all_values):

        col_name = f"{column}_{value.replace(' ', '_').replace('/', '_')}"
        

        df[col_name] = df[column].apply(
            lambda x: 1 if pd.notna(x) and value in str(x).split(',') else 0
        )
    

    df = df.drop(columns=[column])
    
    return df, all_values



def extract_all_values(df: pd.DataFrame, column: str) -> set:
    """Extrait toutes les valeurs uniques d'une colonne."""
    all_values = set()
    for val in df[column].dropna():
        if pd.notna(val) and val != 'N,A':
            values = [v.strip() for v in str(val).split(',')]
            all_values.update(values)
    return all_values


urban_values_train = extract_all_values(x_train, 'urban_type')
urban_values_test = extract_all_values(test_df, 'urban_type')
urban_values_all = urban_values_train | urban_values_test  

geo_values_train = extract_all_values(x_train, 'geography_type')
geo_values_test = extract_all_values(test_df, 'geography_type')
geo_values_all = geo_values_train | geo_values_test

print("🔍 Analyse des valeurs:")
print(f"urban_type - Train: {len(urban_values_train)}, Test: {len(urban_values_test)}, Total: {len(urban_values_all)}")
print(f"  Uniquement dans train: {urban_values_train - urban_values_test}")
print(f"  Uniquement dans test: {urban_values_test - urban_values_train}")

print(f"\ngeography_type - Train: {len(geo_values_train)}, Test: {len(geo_values_test)}, Total: {len(geo_values_all)}")
print(f"  Uniquement dans train: {geo_values_train - geo_values_train.intersection(geo_values_test) if len(geo_values_train - geo_values_test) <= 5 else '(trop pour afficher)'}")
print(f"  Uniquement dans test: {geo_values_test - geo_values_test.intersection(geo_values_train) if len(geo_values_test - geo_values_train) <= 5 else '(trop pour afficher)'}")


🔍 Analyse des valeurs:
urban_type - Train: 5, Test: 5, Total: 5
  Uniquement dans train: set()
  Uniquement dans test: set()

geography_type - Train: 11, Test: 11, Total: 11
  Uniquement dans train: set()
  Uniquement dans test: set()


In [29]:
#multi-hot encoding on train
x_train, _ = multi_hot_encode(x_train, 'urban_type', urban_values_all)
x_train, _ = multi_hot_encode(x_train, 'geography_type', geo_values_all)
print(f"Train shape: {x_train.shape}")

#multi-hot encoding on test
x_test, _ = multi_hot_encode(x_test, 'urban_type', urban_values_all)
x_test, _ = multi_hot_encode(x_test, 'geography_type', geo_values_all)
print(f"Test shape: {x_test.shape}")

Train shape: (296146, 95)
Test shape: (120526, 95)
Test shape: (120526, 95)


### Drop additional columns

In [30]:
cols_to_drop = ["delta_sorted_0_1_days","delta_sorted_1_2_days", "delta_sorted_2_3_days", "delta_sorted_3_4_days","index"]


cols_to_drop_train = [col for col in cols_to_drop if col in x_train.columns]
cols_to_drop_test = [col for col in cols_to_drop if col in x_test.columns]

x_train = x_train.drop(columns=cols_to_drop_train)
x_test = x_test.drop(columns=cols_to_drop_test)

### Imputation

In [ ]:

inf_count_train = np.isinf(x_train.select_dtypes(include=[np.number])).sum().sum()
inf_count_test = np.isinf(x_test.select_dtypes(include=[np.number])).sum().sum()
print(f"Train: {inf_count_train} valeurs infinies")
print(f"Test: {inf_count_test} valeurs infinies")


x_train = x_train.replace([np.inf, -np.inf], np.nan)
x_test = x_test.replace([np.inf, -np.inf], np.nan)


from sklearn.impute import SimpleImputer

numerical_cols_final = x_train.select_dtypes(include=["number", "bool"]).columns.to_list()
categorical_cols_final = x_train.select_dtypes(include=["object", "category"]).columns.to_list()



numerical_imputer = SimpleImputer(strategy='median')
categorical_imputer = SimpleImputer(strategy='most_frequent')


if len(numerical_cols_final) > 0:
    x_train[numerical_cols_final] = numerical_imputer.fit_transform(x_train[numerical_cols_final])
if len(categorical_cols_final) > 0:
    x_train[categorical_cols_final] = categorical_imputer.fit_transform(x_train[categorical_cols_final])


if len(numerical_cols_final) > 0:
    x_test[numerical_cols_final] = numerical_imputer.transform(x_test[numerical_cols_final])
if len(categorical_cols_final) > 0:
    x_test[categorical_cols_final] = categorical_imputer.transform(x_test[categorical_cols_final])


# <u> LightGBM Modele training <u/>

In [ ]:
from lightgbm import LGBMClassifier
import lightgbm as lgb
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score, classification_report, confusion_matrix

### Cross-Validation

In [ ]:
X = x_train.values
y = y_train.values

classes_global = np.unique(y)
num_class = len(classes_global)

cv = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=42)

tr_macro_list, tr_weight_list = [], []
va_macro_list, va_weight_list = [], []

for fold, (tr_idx, va_idx) in enumerate(cv.split(X, y), start=1):
    print(f"\n{'='*60}")
    print(f"Fold {fold}/3")
    print(f"{'='*60}")
    
    X_tr, y_tr = X[tr_idx], y[tr_idx]
    X_va, y_va = X[va_idx], y[va_idx]


    w = compute_class_weight(class_weight="balanced", classes=np.unique(y_tr), y=y_tr)
    sample_weights = np.array([w[int(label)] for label in y_tr])


    model = XGBClassifier(
        objective='multi:softmax',
        num_class=num_class,
        n_estimators=1500,
        learning_rate=0.05,
        max_depth=6,
        min_child_weight=1,
        subsample=0.9,
        colsample_bytree=0.9,
        reg_alpha=0.0,
        reg_lambda=1.0,
        random_state=42,
        n_jobs=-1,
        eval_metric='mlogloss',
        early_stopping_rounds=100,
        verbosity=0
    )

    model.fit(
        X_tr, y_tr,
        sample_weight=sample_weights,
        eval_set=[(X_va, y_va)],
        verbose=False
    )

    pred_tr = model.predict(X_tr)
    pred_va = model.predict(X_va)

    tr_macro = f1_score(y_tr, pred_tr, average="macro")
    tr_weight = f1_score(y_tr, pred_tr, average="weighted")
    va_macro = f1_score(y_va, pred_va, average="macro")
    va_weight = f1_score(y_va, pred_va, average="weighted")

    print(f"Train macro F1:    {tr_macro:.4f}")
    print(f"Train weighted F1: {tr_weight:.4f}")
    print(f"Val macro F1:      {va_macro:.4f}")
    print(f"Val weighted F1:   {va_weight:.4f}")

    tr_macro_list.append(tr_macro)
    tr_weight_list.append(tr_weight)
    va_macro_list.append(va_macro)
    va_weight_list.append(va_weight)


print("\n" + "="*60)
print("CROSS-VALIDATION RESULTS")
print("="*60)
print(f"Train macro   = {np.mean(tr_macro_list):.4f} ± {np.std(tr_macro_list):.4f}")
print(f"Train weighted= {np.mean(tr_weight_list):.4f} ± {np.std(tr_weight_list):.4f}")
print(f"Val   macro   = {np.mean(va_macro_list):.4f} ± {np.std(va_macro_list):.4f}")
print(f"Val   weighted= {np.mean(va_weight_list):.4f} ± {np.std(va_weight_list):.4f}")


Fold 1/3
Train macro F1:    0.8740
Train weighted F1: 0.8289
Val macro F1:      0.5570
Val weighted F1:   0.7677

Fold 2/3
Train macro F1:    0.8740
Train weighted F1: 0.8289
Val macro F1:      0.5570
Val weighted F1:   0.7677

Fold 2/3
Train macro F1:    0.8719
Train weighted F1: 0.8298
Val macro F1:      0.5551
Val weighted F1:   0.7655

Fold 3/3
Train macro F1:    0.8719
Train weighted F1: 0.8298
Val macro F1:      0.5551
Val weighted F1:   0.7655

Fold 3/3
Train macro F1:    0.8754
Train weighted F1: 0.8304
Val macro F1:      0.5585
Val weighted F1:   0.7663

CROSS-VALIDATION RESULTS
Train macro   = 0.8738 ± 0.0014
Train weighted= 0.8297 ± 0.0006
Val   macro   = 0.5569 ± 0.0014
Val   weighted= 0.7665 ± 0.0009
Train macro F1:    0.8754
Train weighted F1: 0.8304
Val macro F1:      0.5585
Val weighted F1:   0.7663

CROSS-VALIDATION RESULTS
Train macro   = 0.8738 ± 0.0014
Train weighted= 0.8297 ± 0.0006
Val   macro   = 0.5569 ± 0.0014
Val   weighted= 0.7665 ± 0.0009
